In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import norm
from tqdm.notebook import tqdm

In [2]:
#work around all those python directory crazyness in order to access the ab_testing module
import os
import sys
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

from ab_testing.stratification import stratified_ttest, get_stratified_statistics

# Effect of relaxed stopping rules on fixed horizon tests

In this notebook we investigate the following question:

When performing a fixed horizon null hypothesis test with relaxed stopping rules (e.g. peeking nd early stopping) what is the effect on the various error rates like false positives, false negatives, sign errors etc.

In [3]:
from decision_rules import ABTestData, FixedHorizonDecisionRule
from simulate import simulate_ab_test_binom

In [4]:
# check 



In [5]:
d = FixedHorizonDecisionRule(data, expected_runtime=14, alpha=.1)

d.t, d.p, d.decision

(-2.0820568340968357, 0.03742794342092418, 'STOP REJECT NULL')

In [7]:
for i in range(100):
    data = simulate_ab_test_binom(n_samples_per_increment = 100, n_increments = 14, mean_a=.5, mean_b=.55)
    d = FixedHorizonDecisionRule(data, expected_runtime=14, alpha=.1)
    print(d.t, d.p, d.decision)

-2.3468619889658484 0.0190016424868825 STOP REJECT NULL
-1.5882472994713743 0.11234341479783976 STOP INCONCLUSIVE
-2.0825048506925485 0.03738701164564331 STOP REJECT NULL
-1.2861039819748505 0.1985131897429053 STOP INCONCLUSIVE
-2.3462252950909193 0.019034069676237192 STOP REJECT NULL
-3.602595876547945 0.00032054144973542537 STOP REJECT NULL
-2.9645027239659343 0.0030573289164386726 STOP REJECT NULL
-4.260039665593522 2.1113642966432374e-05 STOP REJECT NULL
-3.1447071462073177 0.0016799690865404954 STOP REJECT NULL
-1.4382932269039739 0.15046271033126896 STOP INCONCLUSIVE
-3.1481913760305313 0.0016601424686628087 STOP REJECT NULL
-3.5673756087964503 0.0003666253899981939 STOP REJECT NULL
-4.438642766663178 9.403083654999179e-06 STOP REJECT NULL
-3.142495922991812 0.0016926646086264944 STOP REJECT NULL
-3.1443135553389134 0.001682222418750043 STOP REJECT NULL
-1.6269801355505953 0.10385391671481914 STOP INCONCLUSIVE
-3.5997108196384757 0.0003241020843576343 STOP REJECT NULL
-3.33811437